In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor as Xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#SKforecast
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import (
    TimeSeriesFold,
    backtesting_forecaster_multiseries,
    grid_search_forecaster_multiseries,
    bayesian_search_forecaster_multiseries
)
from skforecast.plot import set_dark_theme
from skforecast.feature_selection import select_features_multiseries

#SKLEARN
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

#MLFLOW
import mlflow
from mlflow.tracking import MlflowClient
import pickle

#Exogenous variables 
import holidays
from vacances_scolaires_france import SchoolHolidayDates

In [5]:
#vacances scolaires
d = SchoolHolidayDates()
dates_vacances=set()
for y in (2023,2024):
    vacances=d.holidays_for_year_and_zone(y, 'C')
    dates_vacances.update(vacances.keys())

#jours feriés
feries = holidays.France(years=['2023','2024'])

In [6]:
data = pd.read_csv("lit_cluster_0.csv",index_col="date_semaine", parse_dates=True)
data=data.asfreq('W') 
list_uf =data.columns.tolist()

data['vacances']= data.index.isin(dates_vacances).astype(int)
data['feries'] = data.index.isin(feries).astype(int)
data["month"] = data.index.month
data["week_of_year"] = data.index.isocalendar().week

exog_list=data.columns.drop(list_uf)

C:\Users\yessi\AppData\Local\Temp\ipykernel_21252\1880449936.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data['vacances']= data.index.isin(dates_vacances).astype(int)
C:\Users\yessi\AppData\Local\Temp\ipykernel_21252\1880449936.py:6: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data['feries'] = data.index.isin(feries).astype(int)


In [7]:
n_samples = len(data)
train_size = int(0.6 * n_samples)  
val_size = int(0.2 * n_samples)  

data_train = data.iloc[:train_size]
data_val = data.iloc[train_size:train_size + val_size]
data_test = data.iloc[train_size + val_size:]

print(f"Total samples: {n_samples}")
print(f"Train: {len(data_train)} samples : {data_train.index[0]} to {data_train.index[-1]}")
print(f"Validation: {len(data_val)} samples : {data_val.index[0]} to {data_val.index[-1]}")
print(f"Test: {len(data_test)} samples : {data_test.index[0]} to {data_test.index[-1]}")

Total samples: 105
Train: 63 samples : 2023-01-01 00:00:00 to 2024-03-10 00:00:00
Validation: 21 samples : 2024-03-17 00:00:00 to 2024-08-04 00:00:00
Test: 21 samples : 2024-08-11 00:00:00 to 2024-12-29 00:00:00


In [ ]:
col="1015"
set_dark_theme()
plt.figure(figsize=(12, 4))
 
data_train[col].plot( label='train')
data_val[col].plot( label='validation')
data_test[col].plot( label='test')
plt.show()

In [8]:
# Create and train ForecasterRecursiveMultiSeries
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = RandomForestRegressor(n_estimators=100, random_state=42),
                 lags               = 52,
                 window_features    = RollingFeatures(stats=['mean', 'mean','mean','std','std','std'], window_sizes=[4, 24, 52, 4, 24, 52]),
                 encoding           = 'ordinal'
             )


In [9]:
cv = TimeSeriesFold(
         steps              = 21,
         initial_train_size = len(data_train),
         refit              = False,
         fixed_train_size   =False
     )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
    forecaster            = forecaster,
    series                = data.iloc[:train_size + val_size][list_uf],
    exog                  = data.iloc[:train_size + val_size][exog_list],
    cv                    = cv,
    levels                = None,
    metric                = 'mean_absolute_error',
    add_aggregated_metric = True
)

print("Backtest metrics")
display(metrics_levels)
print("")
print("Backtest predictions")
backtest_predictions.head(4)

╭────────────────────────────────── DataTypeWarning ───────────────────────────────────╮
│ `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning   │
│ models do not allow other types of values. Fitting the forecaster may fail.          │
│                                                                                      │
│ Category : DataTypeWarning                                                           │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\utils\utils.py:63 │
│ 8                                                                                    │
│ Suppress : warnings.simplefilter('ignore', category=DataTypeWarning)                 │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

  0%|          | 0/1 [00:00<?, ?it/s]

Backtest metrics


,levels,mean_absolute_error
0,1015,0.707438
1,1022,0.021361
2,1023,0.122313
3,1026,0.965593
4,1027,2.319720
...,...,...
168,1605,0.330408
169,2431,2.477899
170,average,1.172004
171,weighted_average,1.172004



Backtest predictions


,level,pred
2024-03-17,1015,1.444921
2024-03-17,1022,0.000000
2024-03-17,1023,0.000000
2024-03-17,1026,0.174286


In [10]:

tscv = TimeSeriesSplit(n_splits=3)
regressor = LGBMRegressor(n_estimators=100, max_depth=5, random_state=15926, verbose=-1)
selector = RFECV(estimator=regressor, step=1, cv=tscv, min_features_to_select=1,scoring='neg_mean_absolute_error')
selected_lags, selected_window_features, selected_exog = select_features_multiseries(
    forecaster      = forecaster,
    selector        = selector,
    series          = data.iloc[:train_size+val_size][list_uf],
    exog            = data.iloc[:train_size+val_size][exog_list],
    select_only     = None,
    force_inclusion = None,
    subsample       = 0.5,
    random_state    = 123,
    verbose         = False,
)

╭────────────────────────────────── DataTypeWarning ───────────────────────────────────╮
│ `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning   │
│ models do not allow other types of values. Fitting the forecaster may fail.          │
│                                                                                      │
│ Category : DataTypeWarning                                                           │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\utils\utils.py:63 │
│ 8                                                                                    │
│ Suppress : warnings.simplefilter('ignore', category=DataTypeWarning)                 │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with featur

In [14]:
stats=[]
window_sizes=[]
for i, value in enumerate(selected_window_features):
    stats.append(selected_window_features[i].split("_")[1])
    window_sizes.append(int(selected_window_features[i].split("_")[2]))

forecaster.set_lags(lags=selected_lags)
forecaster.set_window_features(window_features=RollingFeatures(stats=stats, window_sizes=window_sizes))

In [15]:
levels = list_uf

# Search space
def search_space(trial):
    search_space = {
        # Core Random Forest parameters
        'n_estimators'      : trial.suggest_int('n_estimators', 100, 1000),
        'max_depth'         : trial.suggest_int('max_depth', 3, 20),
        'min_samples_split' : trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf'  : trial.suggest_int('min_samples_leaf', 1, 10),
        
        # Feature selection parameters
        'max_features'      : trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'max_samples'       : trial.suggest_float('max_samples', 0.5, 1.0),
        
        # Tree building parameters
        'bootstrap'         : trial.suggest_categorical('bootstrap', [True, False]),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.1),
        'max_leaf_nodes'    : trial.suggest_int('max_leaf_nodes', 10, 1000),
        
        # Complexity control
        'ccp_alpha'         : trial.suggest_float('ccp_alpha', 0.0, 0.1),
    }

    return search_space

cv = TimeSeriesFold(
         steps              = 21,
         initial_train_size = len(data_train),
         refit              = True,
         fixed_train_size=False
     )

results, best_trial = bayesian_search_forecaster_multiseries(
    forecaster       = forecaster,
    series           = data.iloc[:train_size+val_size][list_uf],
    exog             = None,
    search_space     = search_space,
    cv               = cv,
    levels           = list_uf,
    metric           = 'mean_absolute_error',
    aggregate_metric = ['weighted_average', 'average', 'pooling'],
    n_trials         = 5
)



  0%|          | 0/5 [00:00<?, ?it/s]

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  8 17 18 26 30 31 40 43 51 52] 
  Parameters: {'n_estimators': 153, 'max_depth': 10, 'min_samples_split': 16, 'min_samples_leaf': 2, 'max_features': None, 'max_samples': 0.8172004792756605, 'bootstrap': True, 'min_weight_fraction_leaf': 0.06110235106775829, 'max_leaf_nodes': 725, 'ccp_alpha': 0.03229589138531782}
  Backtesting metric: 0.7916642070777891
  Levels: ['1015', '1022', '1023', '1026', '1027', '1035', '1037', '1039', '1042', '1043', '...', '3117', '3118', '3120', '3416', '3421', '9050', '1211', '1308', '1605', '2431']



In [17]:
forecaster.fit(
    series = data.iloc[:train_size+val_size][list_uf],
    exog= None,
    store_in_sample_residuals=True
)

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1191                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\recursive\_foreca │
│ ster_recursive_multiseries.py:1213                                                   │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
col="1015"
set_dark_theme()
plt.figure(figsize=(12, 4))
 
data_train[col].plot( label='train')
data_val[col].plot( label='validation')
data_test[col].plot( label='test')
preds[preds['level']==col]['pred'].plot(label='prediction')
plt.show()

In [22]:
mlflow.set_experiment("Default")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

params = forecaster.regressor.get_params()
params['lags'] = forecaster.lags
params['window features'] = forecaster.window_features
params['exogs'] = forecaster.exog_names_in_
params['regressor'] = forecaster.regressor.__class__.__name__

with mlflow.start_run(run_name='RF_final_c0') as run:
    mlflow.log_params(params)
    mlflow.log_metrics({"MMAE" : mmae, "RMSE": mrmse, "Native MAE": np.mean(naive_mae),"native RMSE": np.mean(naive_rsme), "Mean MAE": np.mean(mean_mae), "Mean RMSE": np.mean(mean_rmse), "Drift MAE": np.mean(drift_mae), "Drift RMSE": np.mean(drift_rmse)})
    # Save forecaster object as pickle
    with open("RF_final_c0.pkl", "wb") as f:
        pickle.dump(forecaster, f)

    # Log the pickle file as an artifact
    mlflow.log_artifact("RF_final_c0.pkl", artifact_path="pickle_folder")

🏃 View run RF_final_c0 at: http://127.0.0.1:5000/#/experiments/0/runs/c24ae0243ea2493c94d9bf8383444317
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


BENCHMARKING

In [19]:
def rmse(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def benchmark_mae(train_series, test_series):

    train_series = pd.Series(train_series)
    test_series = pd.Series(test_series)
    
    naive_preds = pd.Series(np.repeat(train_series.iloc[-1], len(test_series)), index=test_series.index)
    
    mean_pred = pd.Series(np.repeat(train_series.mean(), len(test_series)), index=test_series.index)
    
    n_train = len(train_series)
    drift = (train_series.iloc[-1] - train_series.iloc[0]) / (n_train - 1)
    drift_preds = []
    for h in range(1, len(test_series) + 1):
        drift_preds.append(train_series.iloc[-1] + h * drift)
    drift_preds = pd.Series(drift_preds, index=test_series.index)
    
    results = {}
    results['naive_mae'] = mean_absolute_error(test_series, naive_preds)
    results['mean_mae'] = mean_absolute_error(test_series, mean_pred)
    results['drift_mae'] = mean_absolute_error(test_series, drift_preds)

    results_rmse={}
    results_rmse['naive_rmse'] = rmse(test_series, naive_preds)
    results_rmse['mean_rmse'] = rmse(test_series, mean_pred)
    results_rmse['drift_rmse'] = rmse(test_series, drift_preds)
    
    return results, results_rmse


In [20]:
naive_mae=[]
mean_mae=[]
drift_mae=[]
naive_rsme=[]
mean_rmse=[]
drift_rmse=[]
for i in list_uf:
    results, results_rmse = benchmark_mae(data.iloc[:train_size+val_size][i], data_test[i])
    
    naive_mae.append(results['naive_mae'])
    mean_mae.append(results['mean_mae'])
    drift_mae.append(results['drift_mae'])
    naive_rsme.append(results_rmse['naive_rmse'])
    mean_rmse.append(results_rmse['mean_rmse'])
    drift_rmse.append(results_rmse['drift_rmse'])

preds = forecaster.predict(steps=21)
rmses=[]
maes=[]
for col in list_uf:
    mae = mean_absolute_error(data_test[col], preds[preds['level']==col]['pred'])
    maes.append(mae)
    uf_rmse=rmse(data_test[col], preds[preds['level']==col]['pred'])
    rmses.append(uf_rmse)

mmae = np.mean(maes)
mrmse = np.mean(rmses)

print(f"Mean Absolute Error for cluster 0: {mmae:.2f}")
print("========================")
print("Naive MAE:", np.mean(naive_mae))
print("Mean MAE:", np.mean(mean_mae))
print("Drift MAE:", np.mean(drift_mae))
print("========================")
print(f"Root Mean Squared Error for cluster 0: {mrmse:.2f}")
print("========================")
print("Naive RMSE:", np.mean(naive_rsme))
print("Mean RMSE:", np.mean(mean_rmse))
print("drift RMSE:", np.mean(drift_rmse))





╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ `last_window` has missing values. Most of machine learning models do not allow       │
│ missing values. Prediction method may fail.                                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\yessi\anaconda3\envs\new_env\Lib\site-packages\skforecast\utils\utils.py:98 │
│ 9                                                                                    │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

Mean Absolute Error for cluster 0: 1.90
Naive MAE: 2.555222088835534
Mean MAE: 1.4747356894565056
Drift MAE: 2.9509868525723544
Root Mean Squared Error for cluster 0: 2.36
Naive RMSE: 3.1666786314106035
Mean RMSE: 2.133637429684564
drift RMSE: 3.5699387470722237


In [24]:
k=0
r=0
diff=[]
diff2=[]
for i in list_uf:
    results, results_rmse = benchmark_mae(data.iloc[:train_size+val_size][i], data_test[i])
    mae = mean_absolute_error(data_test[col], preds[preds['level']==col]['pred'])
    uf_rmse=rmse(data_test[col], preds[preds['level']==col]['pred'])

    if mae < results["drift_mae"] and mae < results["mean_mae"] and  mae < results["naive_mae"]:
        k=k+1
        diff.append(abs(mae - results["mean_mae"]))
    if uf_rmse < results_rmse["drift_rmse"] and uf_rmse < results_rmse["mean_rmse"] and  uf_rmse < results_rmse["naive_rmse"]:
        r+=1
        diff2.append(abs(uf_rmse - results_rmse["mean_rmse"]))
print("Number of UFS where model performed better (mae): ",k,":", k/len(list_uf)*100,"%")
print("mean improvement (mae): ", np.mean(diff) if diff else 0)
print("Number of UFS where model performed better (rmse): ", r,":", r/len(list_uf)*100,"%")
print("mean improvement (rmse): ", np.mean(diff2) if diff2 else 0)

Number of UFS where model performed better (mae):  91 : 53.529411764705884 %
mean improvement (mae):  1.611540081955815
Number of UFS where model performed better (rmse):  107 : 62.94117647058823 %
mean improvement (rmse):  2.3940578128493484
